### Translation for kor → eng

In [1]:
import sys
sys.path.insert(0, '../attention-is-all-you-need/')
sys.path.insert(0, '../')

In [4]:
import sentencepiece as spm
import random
import numpy as np
from setting import *
from dataset import load_dataset

In [5]:
sent_pairs = load_dataset(path='/heavy_data/jkfirst/workspace/git/LaH/dataset/')

Exception: 187th sentence of /heavy_data/jkfirst/workspace/git/LaH/dataset/Corpus10/cekcorpus31.txt: 'cp949' codec can't decode byte 0xc9 in position 7412: illegal multibyte sequence
>> 61094 sentence pairs loaded
>> 62094 sentence pairs loaded


In [6]:
sent_pairs[0]

['우리는 춘부장께 도움을 청해야 한다', 'we must ask help from your father']

In [7]:
src_vocab_size = 5000
trg_vocab_size = 3000
src_prefix = 'spm-src-{}'.format(src_vocab_size)
trg_prefix = 'spm-trg-{}'.format(trg_vocab_size)
src_prefix, trg_prefix

('spm-src-5000', 'spm-trg-3000')

In [8]:
src_cmd = templates.format(
    src_input_file,
    pad_id,
    bos_id,
    eos_id,
    unk_id,
    src_prefix,
    src_vocab_size,
    character_coverage,
    model_type)
src_cmd

'--input=./spm_src.txt --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3 --model_prefix=spm-src-5000 --vocab_size=5000 --character_coverage=1.0 --model_type=unigram'

In [9]:
trg_cmd = templates.format(
    trg_input_file,
    pad_id,
    bos_id,
    eos_id,
    unk_id,
    trg_prefix,
    trg_vocab_size,
    character_coverage,
    model_type)
trg_cmd

'--input=./spm_trg.txt --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3 --model_prefix=spm-trg-3000 --vocab_size=3000 --character_coverage=1.0 --model_type=unigram'

In [10]:
spm.SentencePieceTrainer.Train(src_cmd)
spm.SentencePieceTrainer.Train(trg_cmd)

True

In [11]:
src_spm = spm.SentencePieceProcessor()
trg_spm = spm.SentencePieceProcessor()
src_spm.Load('{}.model'.format(src_prefix)) 
trg_spm.Load('{}.model'.format(trg_prefix))

True

In [34]:
with open('{}.vocab'.format(src_prefix), encoding='utf-8') as f:
    src_vocab = [doc.strip() for doc in f]
src_vocab[:10]

['<pad>\t0',
 '<s>\t0',
 '</s>\t0',
 '<unk>\t0',
 '▁\t-2.86683',
 '을\t-3.56501',
 '의\t-3.78798',
 '를\t-3.89044',
 '는\t-4.03717',
 '가\t-4.09034']

In [35]:
with open('{}.vocab'.format(trg_prefix), encoding='utf-8') as f:
    trg_vocab = [doc.strip() for doc in f]
trg_vocab[:10]

['<pad>\t0',
 '<s>\t0',
 '</s>\t0',
 '<unk>\t0',
 '▁the\t-2.87699',
 's\t-3.42794',
 '▁to\t-3.7658',
 '▁a\t-3.80983',
 '▁he\t-4.07266',
 'ed\t-4.09187']

In [36]:
idx = np.random.randint(0, len(sent_pairs))
idx
src_spm.EncodeAsPieces(sent_pairs[idx][0])

['▁질',
 '에',
 '▁있어서',
 '▁우리는',
 '▁다른',
 '▁상',
 '표',
 '의',
 '▁상품',
 '은',
 '▁우리',
 '의',
 '▁것과',
 '▁비교',
 '할',
 '▁수',
 '▁없다',
 '고',
 '▁생각합니다']

In [37]:
idx = np.random.randint(0, len(sent_pairs))
trg_spm.EncodeAsPieces(sent_pairs[idx][1])

['▁the', '▁woman', '▁can', '▁out', 's', 'we', 'ar', '▁her', '▁husband']

In [38]:
idx = np.random.randint(0, len(sent_pairs))
src_sent = trg_spm.EncodeAsPieces(sent_pairs[idx][0])
trg_sent = trg_spm.EncodeAsPieces(sent_pairs[idx][1])
src_sent, trg_sent

(['▁',
  '그는',
  '▁',
  '상류계층에서',
  '▁',
  '활동하',
  '여',
  '▁',
  '교제하는',
  '▁',
  '사람은',
  '▁',
  '모두',
  '▁',
  '귀족',
  '이',
  '다'],
 ['▁he',
  '▁move',
  's',
  '▁in',
  '▁very',
  '▁ra',
  're',
  'f',
  'ied',
  '▁circle',
  's',
  '▁his',
  '▁friends',
  '▁are',
  '▁all',
  '▁lord',
  's'])

In [48]:
n_max_list = [10000, 20000, 30000, 40000, 50000, 60000, 63000]

In [53]:
for n_max in n_max_list:
    # read files
    with open(src_input_file) as f:
        sents = []
        for oneline in f:
            sents.append(oneline.rstrip())
    
    # shuffle and collect sents
    random.shuffle(sents)
    sents = sents[:n_max]
    
    # make vocab
    src_word = {}
    for s in sents:
        s = s.split()
        for si in s:
            try:
                src_word[si] += 1
            except KeyError:
                src_word[si] = 1
                
    print('{} sentences -> {} words'.format(n_max, len(src_word)))

10000 sentences -> 22747 words
20000 sentences -> 36729 words
30000 sentences -> 48369 words
40000 sentences -> 58464 words
50000 sentences -> 67386 words
60000 sentences -> 75547 words
63000 sentences -> 77264 words


In [54]:
for n_max in n_max_list:
    # read files
    with open(trg_input_file) as f:
        sents = []
        for oneline in f:
            sents.append(oneline.rstrip())
    
    # shuffle and collect sents
    random.shuffle(sents)
    sents = sents[:n_max]
    
    # make vocab
    trg_word = {}
    for s in sents:
        s = s.split()
        for si in s:
            try:
                trg_word[si] += 1
            except KeyError:
                trg_word[si] = 1
                
    print('{} sentences -> {} words'.format(n_max, len(trg_word)))

10000 sentences -> 10265 words
20000 sentences -> 15302 words
30000 sentences -> 19130 words
40000 sentences -> 22457 words
50000 sentences -> 25282 words
60000 sentences -> 27831 words
63000 sentences -> 28369 words


In [57]:
src_word['나'], trg_word['am']

(68, 503)